# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull.

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it.

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link.

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell.

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. _Make sure to answer every question marked with a `Q:` for full credit._


# Importing Libraries


In [1]:
import os
import datetime
import re
import shutil

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [2]:
# Use this cell for any import statements you add

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways.


In [3]:
artists = {
    # "robyn": "https://www.azlyrics.com/r/robyn.html",
    # "cher": "https://www.azlyrics.com/c/cher.html",
    "blackpink": "https://www.azlyrics.com/b/blackpink.html",
    "bini": "https://www.azlyrics.com/b/bini.html",
}
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).)

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again.

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages.

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know?

A: Based on the `robots.txt` page, there are four paths that we are not allowed to scrape: `/lyricsdb/`, `/song/`, `/search/`, and `/suggest/`. Since we are scraping information from the artists page, `/r/` and `/c/`, this is allowed.


In [4]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items():
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10 * random.random())

    # now extract the links to lyrics pages from this page
    soup = BeautifulSoup(r.text, "html.parser")
    album_items = soup.find_all("div", class_="listalbum-item")
    links = []  # initialize links list
    for item in album_items:
        a_tag = item.find("a")
        if a_tag and a_tag.get("href"):
            links.append(a_tag["href"])
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links.
    lyrics_pages[artist].extend(links)

Let's make sure we have enough lyrics pages to scrape.


In [5]:
for artist, lp in lyrics_pages.items():
    assert len(set(lp)) > 20

In [6]:
# Let's see how long it's going to take to pull these lyrics
# if we're waiting `5 + 10*random.random()` seconds
for artist, links in lyrics_pages.items():
    print(f"For {artist} we have {len(links)}.")
    print(
        f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours."
    )

For blackpink we have 68.
The full pull will take for this artist will take 0.19 hours.
For bini we have 49.
The full pull will take for this artist will take 0.14 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part.

1. Create an empty folder in our repo called "lyrics".
1. Iterate over the artists in `lyrics_pages`.
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages.
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name.


In [7]:
def generate_filename_from_link(link):

    if not link:
        return None

    # drop the http or https and the html
    name = link.replace("https", "").replace("http", "")
    name = name.replace(".html", "")

    name = name.replace("/lyrics/", "")

    # Replace useless characters with UNDERSCORE
    name = name.replace("://", "").replace(".", "_").replace("/", "_")

    # tack on .txt
    name = name + ".txt"

    return name

In [8]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics"):
    print("Deleting old directory... ")
    shutil.rmtree("lyrics/")

print("Creating `lyrics` directory.")
os.mkdir("lyrics")

Deleting old directory... 
Creating `lyrics` directory.


In [9]:
url_stub = "https://www.azlyrics.com"
start = time.time()


MAX_REQUESTS = None  # limit requests during testing; use None to remove limit
total_pages = 0
total_errors = 0

for artist in lyrics_pages:
    # initialize variables for artist summary
    artist_pages = 0
    artist_errors = 0

    # Use this space to carry out the following steps:

    # 1. Build a subfolder for the artist
    artist_dir = os.path.join("lyrics", artist)
    # Check if lyrics folder exist; delete and recreate
    if os.path.isdir(os.path.join("lyrics", artist)):
        print("Deleting old artist directory...")
        shutil.rmtree(os.path.join("lyrics", artist))
    print(f"Creating artist directory: {artist_dir}")
    os.mkdir(artist_dir)

    # 2. Iterate over the lyrics pages
    # print(lyrics_pages[artist])
    for page in lyrics_pages[artist][:MAX_REQUESTS]:
        lyric_page_url = url_stub + page

        # 3. Request the lyrics page.
        # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request
        # ChatGPT was used to create try-except block to prevent crash
        # for invalid URLs
        try:
            r = requests.get(lyric_page_url, timeout=20)
            time.sleep(5 + 10 * random.random())
        except requests.RequestException as e:
            print(f"Skipping {lyric_page_url} - {e}")
            artist_errors += 1
            total_errors += 1
            continue

        # check if URL returns valid status code
        if r.status_code != 200:
            print(f"There is an error with the URL: {lyric_page_url}")
            print("Skipping...")
            artist_errors += 1
            total_errors += 1
            continue

        # 4. Extract the title and lyrics from the page.
        soup = BeautifulSoup(r.text, "html.parser")
        container = soup.find("div", class_="col-xs-12 col-lg-8 text-center")

        # Extract title
        ringtone_div = container.find("div", class_="ringtone")
        song_title = ringtone_div.find_next("b")
        title = song_title.get_text(strip=True)

        # Extract lyrics
        lyrics_div = ringtone_div.find_next_sibling("div")
        lyrics = lyrics_div.get_text(separator="\n").strip()

        # 5. Write out the title, two returns ('\n'), and the lyrics.
        # Use `generate_filename_from_url` to generate the filename.
        title_and_lyrics = f"{title}\n\n{lyrics}"

        filename = generate_filename_from_link(lyric_page_url)

        filepath = os.path.join(artist_dir, filename)

        # write lyrics to file
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(title_and_lyrics)

        artist_pages += 1
        total_pages += 1

    print(f"{artist}: Saved: {artist_pages}, Errors: {artist_errors}")
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist

print(f"Total invalid URLs: {total_errors}\n")
print(f"Total pages saved: {total_pages}")

Creating artist directory: lyrics/blackpink
Skipping https://www.azlyrics.comhttps://www.azlyrics.com/lyrics/dualipa/kissandmakeup.html - HTTPSConnectionPool(host='www.azlyrics.comhttps', port=443): Max retries exceeded with url: /www.azlyrics.com/lyrics/dualipa/kissandmakeup.html (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x77ca74f4e2d0>: Failed to resolve 'www.azlyrics.comhttps' ([Errno -2] Name or service not known)"))
Skipping https://www.azlyrics.comhttps://www.azlyrics.com/lyrics/jennie/solo.html - HTTPSConnectionPool(host='www.azlyrics.comhttps', port=443): Max retries exceeded with url: /www.azlyrics.com/lyrics/jennie/solo.html (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x77ca75656950>: Failed to resolve 'www.azlyrics.comhttps' ([Errno -2] Name or service not known)"))
blackpink: Saved: 66, Errors: 2
Creating artist directory: lyrics/bini
Skipping https://www.azlyrics.comhttps://www.azlyrics.com/lyrics/sb19/k

In [10]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.33 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com. After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.


In [11]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text):
    return re.findall(r"\w+", text.lower())

## Checking Lyrics

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work.


In [12]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders:
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if "txt" in f or "csv" in f or "tsv" in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files:
        with open("lyrics/" + artist + "/" + f_name) as infile:
            artist_words.extend(words(infile.read()))

    print(
        f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique."
    )

For blackpink we have 61 files.
For blackpink we have roughly 55455 words, 3898 are unique.
For bini we have 47 files.
For bini we have roughly 19687 words, 2252 are unique.
